# 1. Introdução

Este notebook demonstra três técnicas clássicas de binarização de imagens em escala de cinza:

- **Limiarização Simples (Global)**
- **Limiarização Adaptativa (Local)**
- **Limiarização de Otsu (Threshold Otimizado)**



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow  # Caso use Colab
from pathlib import Path

In [ ]:
# Verifica se já foram baixadas as imagens do drive, baixando-as e descompactando se necessário
! [ ! -d f"/content/certidao4.jpg" ] && gdown -O /content/certidao4.jpg "1AcjCzWyIyPmr_-yzAo9WiZbgZyFKjf94"

image_path = Path("/content/certidao4.jpg")



In [ ]:
# Carregando imagem de exemplo
img = cv2.imread('certidao4.jpg', cv2.IMREAD_GRAYSCALE)
cv2_imshow(img)

# 2. Tipos de Limiarização

## 2.1. Limiarização Simples

A limiarização simples utiliza um valor fixo $T$ para decidir se um pixel será branco ou preto. Seja $I(x, y)$ a intensidade do pixel na coordenada $(x, y)$:

$$
B(x, y) = \begin{cases}
255 & \text{se } I(x, y) > T \\
0 & \text{caso contrário}
\end{cases}
$$

Esse processo é chamado de **binarização global**, pois $T$ é constante para toda a imagem.

In [ ]:
def limiarizacao_simples_manual(img, T=150, maxVal=255):
    resultado = np.zeros_like(img)
    resultado[img > T] = maxVal
    return resultado

manual_th = limiarizacao_simples_manual(img)
cv2_imshow(manual_th)

In [ ]:
_, opencv_th = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)
cv2_imshow(opencv_th)

## 2.2. Limiarização Adaptativa

O threshold é calculado localmente com base na vizinhança. Abaixo, criamos uma **versão manual simples** baseada na média local.

Nesta abordagem, o limiar $T(x, y)$ é calculado com base na vizinhança local de cada pixel. Por exemplo, usando a média local:

$$
T(x, y) = \frac{1}{N} \sum_{(i,j) \in \mathcal{N}(x,y)} I(i,j) - C
$$

Onde:
- $\mathcal{N}(x,y)$ é a vizinhança (janela) centrada em $(x,y)$.
- $C$ é uma constante subtraída para ajustar a sensibilidade.

Esse método é robusto à variação de iluminação.

In [ ]:
def limiarizacao_adaptativa_manual(img, block_size, C):
    padded = cv2.copyMakeBorder(img, block_size//2, block_size//2,
                                block_size//2, block_size//2, cv2.BORDER_REPLICATE)
    out = np.zeros_like(img)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            bloco = padded[i:i+block_size, j:j+block_size]
            t = np.mean(bloco) - C
            out[i,j] = 255 if img[i,j] > t else 0
    return out

#Escolha um block_size e C para obter o resultado desejado
adapt_manual = limiarizacao_adaptativa_manual(img)
cv2_imshow(adapt_manual)

In [ ]:
# descubra como fazer o mesmo com OpenCV

## 2.3. Limiarização de Otsu

O método de Otsu determina automaticamente o melhor threshold $T^*$ que **minimiza a variância intra-classe** ou **maximiza a separação entre classes** no histograma.

A variância entre classes para um threshold $t$ é dada por:
$$
\sigma_b^2(t) = \omega_0(t) \omega_1(t) [\mu_0(t) - \mu_1(t)]^2
$$
Onde:
- $\omega_0$, $\omega_1$: probabilidades das classes (áreas do histograma à esquerda e à direita do threshold)
- $\mu_0$, $\mu_1$: médias das classes

A ideia é testar todos os thresholds possíveis (0–255) e escolher aquele que **maximiza $\sigma_b^2(t)$**.

O algoritmo de Otsu encontra **automaticamente** o valor de threshold que minimiza a variância intra-classe. Abaixo comparamos o histograma e a imagem binarizada.

In [ ]:
# Implementação simplificada do método de Otsu
def otsu_manual(img):
    # 1. Calcula o histograma da imagem (256 bins para 256 tons de cinza)
    hist = cv2.calcHist([img], [0], None, [256], [0, 256]).ravel()

    # 2. Total de pixels da imagem
    total = img.shape[0] * img.shape[1]

    # 3. Soma total ponderada das intensidades (usada para média global)
    sum_total = np.dot(np.arange(256), hist)

    # Inicialização das variáveis
    sumB = 0         # Soma ponderada da classe background (fundo)
    wB = 0           # Peso (número de pixels) da classe fundo
    max_var = 0      # Variância entre classes máxima encontrada
    threshold = 0    # Threshold ideal (t) que maximiza a variância entre classes

    # 4. Varre todos os possíveis valores de threshold (0 a 255)
    for t in range(256):
        wB += hist[t]             # Atualiza peso da classe fundo
        if wB == 0:
            continue              # Ignora se fundo vazio
        wF = total - wB           # Peso da classe frente (complementar)
        if wF == 0:
            break                # Se frente vazio, encerra

        sumB += t * hist[t]       # Atualiza soma ponderada do fundo
        mB = sumB / wB            # Média da classe fundo
        mF = (sum_total - sumB) / wF  # Média da classe frente

        # 5. Calcula variância entre classes para o threshold t
        var_between = wB * wF * (mB - mF) ** 2

        # 6. Atualiza se a variância for maior que o máximo atual
        if var_between > max_var:
            max_var = var_between
            threshold = t

    # 7. Aplica o threshold ótimo encontrado para gerar a imagem binarizada
    manual_otsu = np.where(img > threshold, 255, 0).astype(np.uint8)

    return manual_otsu, threshold  # Retorna a imagem binária e o threshold usado

# Uso da função com imagem de entrada (tons de cinza)
otsu_manual_img, t_manual = otsu_manual(img)

# Exibe o resultado (OpenCV no Colab)
cv2_imshow(otsu_manual_img)
